In [1]:


import json
def prepare_label(dict_data)-> list:
    """
        dict_data : { e_id: {
                            'log_features':[] ,
                            'info_features':[] ,
                            'label' : 1 or 0
                            } 
                    }

    Returns:
        list_assemble_data : [[ *log_features , *info_features ],......]
        list_e_id : [ e_id_1 ,e_id_2,...... ]
        list_label : [ 0, 1, ..........]
    """    
    list_assemble_data = []
    list_e_id = []
    list_label = []
    for e_id,dict_ in dict_data.items():

        log_data  = dict_['features']

        label     = dict_['label']

        list_assemble_data.append(log_data)
        list_e_id.append(int(e_id))
        list_label.append(int(label))

    return list_assemble_data , list_e_id , list_label

dict_train=json.load(open('Final_Dataset\\transfor_matrix_train.json','r'))
dict_test = json.load(open('Final_Dataset\\transfor_matrix_test.json','r'))

In [4]:
dict_eid_scene = {}
video_      = ['11','12','13','14','15']
problem_    = ['21','22','23','24','25','26']
comment_    = ['31','32','33','34']
courseware_ = ['41','42','43','44','45','46','47']
scene_dict    = {}
for start in [*video_,*problem_,*comment_,*courseware_]:

    for end in [*video_,*problem_,*comment_,*courseware_]:
        scene = str(start)+str(end)
        scene_dict[ scene ] = 0

In [7]:
t = json.load(open('after_processed_data_file\\transfor_matrix_test.json','r'))

In [18]:

def nomilized_matrix(dict_matrix)-> dict:
    for eid,dict_ in dict_matrix.items():
        matrix_= dict_['features'][18:]
        sum_ = np.sum(matrix_)
        for i in range(18,502):
            dict_matrix[eid]['features'][i] /=sum_
    
    return dict_matrix
dict_test_nom = nomilized_matrix(dict_test)


In [20]:
dict_train_nom = nomilized_matrix(dict_train)

d:\zyh\_workspace\dropout_prediction\backup\transfor_matrix_advance.py:6: RuntimeWarning: invalid value encountered in long_scalars
  transfor_matrix:str,


In [21]:
list_data_train,list_e_id_train,list_label_train = prepare_label(dict_train_nom)
list_data_test,list_e_id_test,list_label_test    = prepare_label(dict_test_nom)


In [22]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
def predict_label_to_int(predict_label_list,threshold):

    predict_label_int = []
    for i in predict_label_list:
        value= i
        if value >threshold:label_ = int(1)
        else:label_ = int(0)
        predict_label_int.append(label_)

    return predict_label_int
def measure(predict_label_int,list_label_test):
    f1             = f1_score(predict_label_int,list_label_test)
   # accuracy = accuracy_score(predict_label_int,list_label_test)
    AUC =       roc_auc_score(predict_label_int,list_label_test)
    # precision = precision_score(predict_label_int,test_label.tolist())
    # recall = recall_score(predict_label_int,test_label.tolist())

    print(round(f1,4),round(AUC,4))

train_data =np.array( list_data_train)
test_data  =np.array( list_data_test)

Padding = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_data_padding = Padding.fit_transform(train_data)
test_data_padding  = Padding.fit_transform(test_data)



In [23]:

import xgboost as xgb
dtrain = xgb.DMatrix(train_data_padding, list_label_train)
dtest = xgb.DMatrix(test_data_padding,list_label_test)
num_rounds = 100
params = {}
watchlist = [(dtrain,'train'),(dtest,'test')]
XGB_nom = xgb.train(
    params, 
    dtrain, 
    num_rounds,
    watchlist,
    early_stopping_rounds=20) 
XGB_predict_label = XGB_nom.predict(dtest)


[0]	train-rmse:0.42291	test-rmse:0.43106
[1]	train-rmse:0.37641	test-rmse:0.39334
[2]	train-rmse:0.35065	test-rmse:0.37356
[3]	train-rmse:0.33660	test-rmse:0.36379
[4]	train-rmse:0.32920	test-rmse:0.35884
[5]	train-rmse:0.32445	test-rmse:0.35692
[6]	train-rmse:0.32176	test-rmse:0.35622
[7]	train-rmse:0.32002	test-rmse:0.35567
[8]	train-rmse:0.31871	test-rmse:0.35510
[9]	train-rmse:0.31784	test-rmse:0.35521
[10]	train-rmse:0.31706	test-rmse:0.35484
[11]	train-rmse:0.31633	test-rmse:0.35494
[12]	train-rmse:0.31565	test-rmse:0.35653
[13]	train-rmse:0.31530	test-rmse:0.35600
[14]	train-rmse:0.31491	test-rmse:0.35665
[15]	train-rmse:0.31436	test-rmse:0.35645
[16]	train-rmse:0.31408	test-rmse:0.35734
[17]	train-rmse:0.31361	test-rmse:0.35825
[18]	train-rmse:0.31340	test-rmse:0.35915
[19]	train-rmse:0.31330	test-rmse:0.36214
[20]	train-rmse:0.31315	test-rmse:0.36247
[21]	train-rmse:0.31291	test-rmse:0.36233
[22]	train-rmse:0.31258	test-rmse:0.36255
[23]	train-rmse:0.31205	test-rmse:0.36251
[2

In [24]:
for i in [0.35,0.4,0.45]:
        
    XGB_predict_label_int = predict_label_to_int(XGB_predict_label,threshold=i)
    measure(
            XGB_predict_label_int,list_label_test)


0.8893 0.7799
0.8871 0.7671
0.8829 0.7543


In [25]:
transfor_node = [
    'seek_video', 'load_video', 'play_video', 'pause_video', 'stop_video', 
    'problem_get', 'problem_check', 'problem_save', 'reset_problem', 'problem_check_correct', 'problem_check_incorrect', 
    'create_thread', 'create_comment', 'delete_thread', 'delete_comment', 
    'click_info', 'click_courseware', 'close_courseware', 'click_about', 'click_forum', 'close_forum', 'click_progress']
feature_dict    = {
     0:'L_mean',1:'L_var',2:'L_skew',3:'L_kurtosis',
     4:'S_mean',5:'S_var',6:'S_skew',7:'S_kurtosis'
    ,8:'gender'
    ,9:'birth_year'
    ,10:'edu_degree'
    ,11:'course_category'
    ,12:'course_type'
    ,13:'course_duration'
    ,14:'student_amount',
    15:'course_amount',
    16:'dropout_rate_of_course',
    17:'dropout_rate_of_user'}
i = 18
for start in transfor_node:
    for end in transfor_node:
        scene = str(start)+'_TO_'+str(end)
        feature_dict[ i ] = scene
        i+=1


In [26]:
a=1

In [27]:
from sklearn import linear_model   
lr = linear_model.LinearRegression(
    normalize=True
    ,n_jobs=-1)
lr.fit(train_data_padding, list_label_train)
result = lr.predict(test_data_padding)

#for i in [0.001, 0.499,0.5,0.501 ]:
for i in [0.001, 0.499,0.5,0.501,0.6,0.7,0.8 ]:
    predict_LinearRegression = predict_label_to_int(
        result,
        threshold= i)
    measure(
        predict_LinearRegression,list_label_test)


0.8614 0.5896
0.8595 0.6504
0.8595 0.6502
0.8595 0.6502
0.859 0.6599
0.86 0.678
0.8587 0.6876


In [ ]:
a=1